In [1]:
import numpy as np
from PIL import Image
import torch
import os
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import vgg
import time
import FlickrDataLoader
import torchvision.transforms as transforms

In [2]:
img_dir = r'./data/img/Flicker8k_Dataset/'
cap_path = r'./data/label/Flickr8k.lemma.token.txt'
train_txt = r'./data/label/Flickr_8k.trainImages.txt'
val_txt = r'./data/label/Flickr_8k.devImages.txt'
test_txt = r'./data/label/Flickr_8k.testImages.txt'

In [3]:
class VggConv(nn.Module):
            def __init__(self, model):
                super(VggConv, self).__init__()
                self.features = nn.Sequential(
                    *list(model.features.children())[:-3]
                )
            def forward(self, x):
                x = self.features(x)
                return x

In [4]:
mytransform = transforms.Compose(
            [
                transforms.Scale((224,224)),
                transforms.RandomHorizontalFlip(),
                # (H x W x C) in the range [0, 255] to (C x H x W) in the range [0.0, 1.0].
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
            ]
        )

In [25]:
flicker8k_val = FlickrDataLoader.Flicker8k(img_dir, cap_path, val_txt, transform=mytransform, train=True)
model = vgg.vgg16_bn(True)
model.eval()
model_conv = VggConv(model).cuda()
bsz = 64

In [77]:
trainloader = torch.utils.data.DataLoader(flicker8k_val, batch_size=bsz,
                                          shuffle=False, num_workers=2)

In [105]:
feat = None
# w2id = {}
# count = 0
caption = []
for i, data in enumerate(trainloader, 0):
    # get the inputs
    inputs, capt = data
    # wrap them in Variable
    inputs = Variable(inputs, volatile=True).cuda()
    
    out_ = model_conv(inputs)

    if feat is None:
        feat = out_.view(bsz,512,-1).transpose(1,2)
    else:
        feat = torch.cat((feat, out_.view(out_.data.shape[0], 512, -1).transpose(1,2)), 0)
    
    for k in range(out_.data.shape[0]):
        cap = []
        for j in range(5):
            tmp = capt[j][k].lower().split(' ')
            a = []
            for l in tmp:
                a.append(w2id[l])
            cap.append(a)
        caption.append(cap)

#     for j in range(5):
#         for k in range(out_.data.shape[0]):
            
#             tmp = capt[j][k].lower().split(' ')
#             for l in tmp:
#                 if l not in w2id:
#                     w2id[l] = count
#                     count += 1

In [92]:
# id2w = {}
# for (w,idx) in w2id.iteritems():
#     id2w[idx] = w

In [97]:
# import pickle
# with open('dict.pkl','w') as f:
#     pickle.dump((w2id, id2w), f)

In [99]:
# with open('feat.pkl','w') as f:
#     pickle.dump(feat.data.cpu().numpy(), f)

In [115]:
# with open('capt.pkl','w') as f:
#     pickle.dump(caption, f)